# LSTM

- ## Preliminaries

- ### Imports

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import os, os.path
from os import listdir
from os.path import isfile, join
from unicodedata import normalize
import re

Using TensorFlow backend.


- ### Check GPU usage

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [3]:
get_available_gpus()

[u'/gpu:0']

----------

- ### Check and set Twitter's API

In [4]:
import twitter
api = twitter.Api(consumer_key='LyNVanTEQEOEGKfXAMeLv6AKG',
                    consumer_secret='0lJvhaaOP5cRZWm6rxwyBIAypd1P7eiDx9f74KBDlLrSldNuBQ',
                    access_token_key='855852332034265088-geTEVmA7xIsOD3WCZyfBNnqjRdS1MhW',
                    access_token_secret='kJMwMl67e3nYrqaGWzIizxzQpRZhtBfOnwPflO1fk3cOt')

In [5]:
print(api.VerifyCredentials())

{"created_at": "Sat Apr 22 18:34:31 +0000 2017", "default_profile": true, "description": "Learning how to be creative", "followers_count": 2, "friends_count": 3, "id": 855852332034265088, "lang": "en", "location": "Somewhere in the cloud", "name": "ArtistBot", "profile_background_color": "F5F8FA", "profile_banner_url": "https://pbs.twimg.com/profile_banners/855852332034265088/1492892354", "profile_image_url": "http://pbs.twimg.com/profile_images/855878764143804417/r55Z2Js5_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "TheTalkativeBot", "status": {"created_at": "Mon Apr 24 13:46:57 +0000 2017", "id": 856504737352601601, "id_str": "856504737352601601", "in_reply_to_screen_name": "dvp_tran", "in_reply_to_status_id": 856504560373911552, "in_reply_to_user_id": 747074580754403328, "lang": "en", "source": "<a href=\"http://www.google.com\" rel=\"nofollow\">TheScenarioBot</a>", "text": "@dvp_tran Automate an

-------------------

# I. Learning from corpus

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [6]:
#Load and concatenate files:

DIR="../../LSTM/data/poemes/"
all_files = [f for f in listdir(DIR) if isfile(join(DIR, f))]

#choose how many files to concatenate:
nb_files=7
if nb_files>len(all_files):
    nb_files=len(all_files)
    
    
out_path="poetry/data/"
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path+"input/"):
    os.makedirs(out_path+"input/")
    
with open(out_path+'input/poemes.txt', 'w') as outfile:
    for fname in all_files[0:nb_files]:
        with open(DIR+fname) as infile:
            for line in infile:
                outfile.write(line)
        print ("Done concatenating file : %s" %fname)

Done concatenating file : poemes1.txt


In [7]:
#load file
file_name=out_path+'input/poemes.txt'
text = open(file_name).read()
text=normalize('NFKD',text.decode('latin1'))

In [8]:
to_delete = u'Skip to main content\nweb\ntexts\nmovies\naudio\nsoftware\nimage\nlogo\nToggle navigation\nsearch\nSearch the Archive\nSearch\n Search\n\nupload\npersonSIGN IN\nFull text of "Quelques poA\u0303 \u0308mes"\nSee other formats\nThis is a digital copy of a book that was preserved for gA\u0303\xa9nA\u0303\xa9rations on library shelves before it was carefully scanned by Google as part of a project \nto make the world\'s books discoverable online. \n\nIt bas survived long enough for the copyright to expire and the book to enter the public domain. A public domain book is one that was never subject \nto copyright or whose lA\u0303\xa9gal copyright term has expired. Whether a book is in the public domain may vary country to country. Public domain books \nare our gateways to the past, representing a wealth of history, culture and knowledge that \'s often difficult to discover. \n\nMarks, notations and other marginalia prA\u0303\xa9sent in the original volume will appear in this file - a reminder of this book\' s long journey from the \npublisher to a library and finally to y ou. \n\nUsage guidelines \n\nGoogle is proud to partner with libraries to digitize public domain materials and make them widely accessible. Public domain books belong to the \npublic and we are merely their custodians. Nevertheless, this work is expensive, so in order to keep providing this resource, we hA\u0303\xa2ve taken steps to \nprevent abuse by commercial parties, including placing technical restrictions on automated querying. \n\nWe also ask that y ou: \n\n+ Make non-commercial use of the files We designed Google Book Search for use by individuals, and we request that you use thA\u0303 \u0308se files for \nPersonal, non-commercial purposes. \n\n+ Refrain from automated querying Do not send automated queries of any sort to Google\'s System: If you are conducting research on machine \ntranslation, optical character rA\u0303\xa9cognition or other areas where access to a large amount of text is helpful, please contact us. We encourage the \nuse of public domain materials for thA\u0303 \u0308se purposes and may be able to help. \n\n+ Maintain attribution The Google "watermark" you see on each file is essential for informing people about this project and helping them find \nadditional materials through Google Book Search. Please do not remove it. \n\n+ Keep it lA\u0303\xa9gal Whatever your use, remember that you are responsible for ensuring that what you are doing is lA\u0303\xa9gal. Do not assume that just \nbecause we believe a book is in the public domain for users in the United States, that the work is also in the public domain for users in other \ncountries. Whether a book is still in copyright varies from country to country, and we can\'t offer guidance on whether any spA\u0303\xa9cifie use of \nany spA\u0303\xa9cifie book is allowed. Please do not assume that a book\'s appearance in Google Book Search means it can be used in any manner \nany where in the world. Copyright infringement liability can be quite severe. \n\nAbout Google Book Search \n\nGoogle\'s mission is to organize the world\'s information and to make it universally accessible and useful. Google Book Search helps readers \ndiscover the world\'s books while helping authors and publishers reach new audiences. You can search through the full text of this book on the web \n\nat http : //books . google . com/| \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\n/>A\u0302\xab, \n\n\n\nQUELQUES POA\u0303\x8bMBS \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nIL A A\u0303 tA\u0303\xa9 TinA\u0303\xa9 DE CET OUVBAGB : \n\nTreize exemplaires Japon impA\u0303\xa9rial, \n\n{dont trois hors commerce) numA\u0303\xa9rotA\u0303\xa9s \n\nde 1 A\u0303 iO et de il A\u0303  iS \n\nVingt exemplaires vA\u0303\xa9lin de Rives \n\n{dont cinq hors commerce) numA\u0303\xa9rotA\u0303\xa9s \n\nde U A\u0303  28 et de 29 k 33 \n\nHuit cent quatre-vingts exemplaires ordinaires \n\n(dont quatre-vingts hors commerce) numA\u0303\xa9rotA\u0303\xa9s \n\nde 34 A\u0303  833 et de 834 A\u0303  9i3, \n\n\n\nNA\u0302\xbb221 \n\n\n\nCopyright by G. CrA\u0303 \u0308s et C", 19f6 \nTous droits de reproduction et d\'adaptation rA\u0303\xa9servA\u0303\xa9s pour tous pays. \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n\nDigitized by \n\n\n\nGoogle \n\n\n'

In [9]:
text = text.replace(to_delete,"").replace('Digitized by',"").replace('Google',"") 
text = re.sub("\n\n+" , "\n", text)

In [10]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 72089
total chars: 108


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [11]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [12]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 64 ms, sys: 20 ms, total: 84 ms
Wall time: 83 ms


** 2. Build the network**

In [13]:
HIDDEN_DIM= 500 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [14]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [15]:
# Generate some sample before training to know how bad it is!
bla = generate_text(model, 100, VOCAB_SIZE, indices_char)
#api.PostUpdate(status=bla[0:123])

«bb)))&B&&&33gggiQ33ggQQ333gg3gQi33ggdddd66E222224444«««««««bbbb33333ggii3ggdddd::4444P4xPPX«««««««

**3. Train network**

In [16]:
def get_iternb(string):
    return re.findall(r'checkpoint_500_epoch_(.*).hdf5', string)[0]

In [18]:
#batch size equals to seq length here
BATCH_SIZE=100
#len of desired output
GENERATE_LENGTH=140
DIR=out_path+"weights/weight_attempt_s01/"
flag=True

try:
    onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]
    iteration=[]
    for files in onlyfiles:
        iteration.append(int(get_iternb(files)))
    iteration=max(iteration)

    last_checkpoint=DIR+onlyfiles[0][0:21]+str(iteration)+'.hdf5'
except Exception as e:
    print(e)
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    nb_files=0

if nb_files>0:
    model.load_weights(last_checkpoint)
else:
    iteration=0
    
print("Starting at iteration : %s" %iteration)
while flag==True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    iteration += 1
    bla=generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if iteration % 50 == 0:
        print("\n\nIteration nb : %s" %iteration)
        #api.PostUpdate(status=bla[0:123])
        model.save_weights(DIR+'checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, iteration))
        #remove unecessary files:
        for files in onlyfiles:
            os.remove(DIR+files)
        onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]

    if iteration>=500:
        print("Stopping...")
        flag=False

Starting at iteration : 450


--------------------
Epoch 1/1
2s - loss: 0.0435
RÃMES 
Entre de morts cratÃ ̈res toujours lumineux, â 
Eternellement calmes, non touchÃ©s de la Jes. 
 
 
142 QUELQUES POÃMES 
Et a

--------------------
Epoch 1/1
2s - loss: 0.0410
T. -A. Hoffmann. â Le Chat-Mourre. 
Charles V.vn Lerberghb. â Les Flaireurs. â PoÃ ̈mes. 
Mais TÃ©nÃ ̈res et pors es nous loin I

--------------------
Epoch 1/1
2s - loss: 0.0432
̄e sang est les 
PÃ©missement l'ordide son pas 
d'ait qu'in couleat, et qui antÃ© d'une jeune des 
tuits veuxtus Ã  peine 
Comme le mysui

--------------------
Epoch 1/1
2s - loss: 0.0478
« la pensÃ©e puissante 1 . . 
Non I le ciel n'est pas lÃ , 
Dans le profond Espace hantÃ© de cauche, un diber. 
Sous seraient une est pas

--------------------
Epoch 1/1
2s - loss: 0.0416
Ã©tait l'ennemi, il en 
est deux fois plus, 
Et qui s'approchent, et muets^ et menaÃ§ants. 
Et la prisÃ©e de lous solident la rÃ©aient l

---------------

**4. Generate text**

In [ ]:
def save_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    if not os.path.exists(out_path+"generate/"):
        os.makedirs(out_path+"generate/")
    with open(out_path+"generate/output.txt","w") as f:
        f.write(('').join(y_char))
    return ('').join(y_char)


In [ ]:
%%time
out = save_text(model, 1500, VOCAB_SIZE, indices_char)